In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [18]:
transform = transforms.ToTensor()

In [19]:
train_data = MNIST(root='data', train=True, transform=transform, download=True)
test_data = MNIST(root='data', train=False, transform=transform, download=True)

In [20]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [21]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 7 * 7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [22]:
model = SimpleNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # images = images.view(images.size(0), -1).to(device)
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item() 

    print(f"Epoch [{epoch+1}/{epochs}], Step [{labels.size(0)}], Loss: {loss.item():.4f}")
    

Epoch [1/5], Step [32], Loss: 0.0556
Epoch [2/5], Step [32], Loss: 0.0488
Epoch [3/5], Step [32], Loss: 0.0831
Epoch [4/5], Step [32], Loss: 0.2064
Epoch [5/5], Step [32], Loss: 0.0406


In [24]:
model.eval()
correct = 0
total = 0

In [25]:

with torch.no_grad():
    for images, labels in test_loader:
        # images = images.view(images.size(0), -1).to(device)
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        predictions = outputs.argmax(dim=1)

        total += labels.size(0)
        correct += (predictions == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')



Accuracy of the model on the test images: 98.74 %


In [26]:
torch.save(model.state_dict(), 'digit_classifier.pth')
print('Model saved to digit_classifier.pth')

Model saved to digit_classifier.pth
